<a href="https://colab.research.google.com/github/malinphy/hate_speech/blob/main/hate_speech_malp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn  as sns
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [ ]:
url = 'https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv'

df = pd.read_csv(url)

 

In [ ]:
df.head(2)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...


In [ ]:
df['tweet'][4]

'!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;'

plt.hist(df['hate_speech'])

plt.hist(df['count'])

plt.hist(df['offensive_language'])

In [ ]:
tweets_mali = df['tweet'].copy()
tweets = df.tweet

In [ ]:
def word_counter(x):
    
    splitter = []
    for i in x:
        var1 = i.split()
        for j in var1:
            splitter.append(j)
            
    word_number_count = Counter(splitter).most_common()
    
    words = []
    words_number = []
    
    for i in word_number_count:
        words.append(i[0])
        words_number.append(i[1])
    
    
    words = words[0:50]
    words_number = words_number[0:50]
    return words,words_number

X,y = word_counter(tweets)
plt.figure(figsize=(15,10))
sns.barplot(X,y, palette="Reds_r")
plt.xticks(rotation=90)
plt.show

In [ ]:
numberOfWords = [] 

for i in range(len(tweets)):
    x = len(tweets_mali[i].split())
    numberOfWords.append(x)

plt.figure(figsize=(12,8))
sns.distplot(numberOfWords, 50  , color="red")
plt.xlabel('Length of a sample')
plt.ylabel('Number of samples')
plt.title('Sample length distribution')
#plt.xlim([-100,6000])

In [ ]:
for i in range(len(tweets)):
    tweets_mali[i] = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', tweets_mali[i]) # regex command to strip url links
    tweets_mali[i] = re.sub('\s+', ' ',tweets_mali[i]) # regex command to rearrange space pattern
    tweets_mali[i] = re.sub('@[\w\-]+','',tweets_mali[i]) # regex command to strip the mentions 
    
    RT_pattern = []
    
for i in range(len(tweets)):
    x = re.findall(":",tweets_mali[i])
    if x == [':']:
        RT_pattern.append(i)
    
for i in (RT_pattern):
    # regex command to strip the RT!! pattern
    tweets_mali[i] = re.split(":",tweets_mali[i])[1]
    
for i in range (len(tweets)):
    tweets_mali[i] = re.sub("!|@|[.]|:|,|[0-9]|#|&|;|[|]|[\"]", "",tweets_mali[i])
    tweets_mali[i] = tweets_mali[i].lower()
    #tweets_mali[i] = re.sub("[0-9],",tweets_mali[i]")

X_clean,y_clean = word_counter(tweets_mali)

plt.figure(figsize=(15,10))
sns.barplot(X_clean,y_clean, palette="Reds_r")
plt.xticks(rotation=90)
plt.show

## BU FONKSIYONDA MEAN DEGERLERINI HESAPLAMAK GEREK DAHA SONRA HESAPLANAN MEAN DEGERLERIYLE GRAFIKTEN ELDE EDILEN 
## PEAK DEGERLERININ YAKINLIGINI KARSILASTIR.


plt.figure(figsize=(12,8))
sns.distplot([len(s) for s in tweets], 40  , 

# This is formatted as code


color="purple")
sns.distplot([len(a) for a in tweets_mali], 40  , color="orange")
plt.xlabel('Length of a sample')
plt.ylabel('Number of samples')
plt.title('Sample length distribution')
plt.xlim([-20,250])
plt.legend(['Raw Data','Processed Data'])

In [ ]:
(np.max(tweets_mali))

'~~ruffled  ntac eileen dahlia - beautiful color combination of pink orange yellow amp white a coll '

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
kerasTK = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',)

kerasTK_corp=kerasTK.fit_on_texts(tweets_mali)


vocab_size  = len(kerasTK.word_index)+1
vocab_size

21571

In [ ]:
uzunluk = 100
total_seq = kerasTK.texts_to_sequences(tweets_mali)
total_seq_pad = pad_sequences(total_seq,maxlen=uzunluk,padding='post')

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
y = df['class'].copy()
from sklearn.preprocessing import LabelBinarizer

class_bin = LabelBinarizer().fit_transform(y)
class_bin

array([[0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [ ]:
X_train,X_test, y_train,y_test = train_test_split(total_seq_pad,class_bin,train_size =0.8,random_state=42)
#y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
#y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

In [ ]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((19826, 100), (4957, 100), (19826, 3), (4957, 3))

In [ ]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense

In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Embedding(vocab_size ,2000,input_length=uzunluk),
                             tf.keras.layers.Dense(20000,activation = 'sigmoid'),
                             tf.keras.layers.Dense(3,activation='softmax')
])

In [ ]:
model.compile(loss= 'hinge',
              metrics=['accuracy'],
              optimizer='adam')

In [ ]:
model.fit(X_train,y_train,epochs= 5)

Epoch 1/5


ValueError: ignored

In [ ]:
res= model.predict(X_test)

m=2

res[m],y_test[m]